In [1]:
import sys
sys.path.append('/openbayes/home/FinRL')
import pandas as pd
import numpy as np
from datetime import datetime
import time
from finrl.apps import config
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/envs/trade/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
bit_data = pd.read_csv("BCHAIN-MKPRU.csv")
gold_data = pd.read_csv("LBMA-GOLD.csv")

In [3]:
def time_transform(time):
    time_array = time.split("/")
    new_time = f"20{int(time_array[2]):02d}-{time_array[0]}-{time_array[1]}"
    return new_time

In [4]:
bit_data.Date = bit_data.Date.apply(time_transform)
gold_data.Date = gold_data.Date.apply(time_transform)

In [5]:
bit_data = bit_data.sort_values(by=['Date']).reset_index(drop=True)
gold_data = gold_data.sort_values(by=['Date']).reset_index(drop=True)

In [6]:
def day_generate(date):
    return datetime.fromtimestamp(int(time.mktime(time.strptime(date,"%Y-%m-%d")))).weekday()

In [7]:
# new cols: date  open  high  low  close  tic  day
bit_data.columns = ['date', 'close']
bit_close_vals = bit_data.close.copy(deep=True)
bit_open_vals = bit_close_vals[:-1]
new_bit_data = bit_data.drop(0).reset_index(drop=True) # !!!
new_bit_data.insert(1, "open", bit_open_vals)
bit_high_vals = new_bit_data.apply(lambda x:max(x["open"], x["close"]), axis=1)
new_bit_data.insert(2, "high", bit_high_vals)
bit_low_vals = new_bit_data.apply(lambda x:min(x["open"], x["close"]), axis=1)
new_bit_data.insert(3, "low", bit_low_vals)
bit_day = new_bit_data.date.apply(day_generate)
new_bit_data.insert(5, "tic", 'bitcoin')
new_bit_data.insert(6, "day", bit_day)
new_bit_data.head(5)

,date,open,high,low,close,tic,day
0,2016-10-10,614.82,617.54,614.82,617.54,bitcoin,0
1,2016-10-11,617.54,617.54,614.77,614.77,bitcoin,1
2,2016-10-12,614.77,635.01,614.77,635.01,bitcoin,2
3,2016-10-13,635.01,635.96,635.01,635.96,bitcoin,3
4,2016-10-14,635.96,635.96,634.02,634.02,bitcoin,4


In [8]:
# new cols: date  open  high  low  close  tic  day
gold_data.columns = ['date', 'close']
gold_close_vals = gold_data.close.copy(deep=True)
gold_open_vals = gold_close_vals[:-1]
new_gold_data = gold_data.drop(0).reset_index(drop=True) # !!!
new_gold_data.insert(1, "open", gold_open_vals)
gold_high_vals = new_gold_data.apply(lambda x:max(x["open"], x["close"]), axis=1)
new_gold_data.insert(2, "high", gold_high_vals)
gold_low_vals = new_gold_data.apply(lambda x:min(x["open"], x["close"]), axis=1)
new_gold_data.insert(3, "low", gold_low_vals)
gold_day = new_bit_data.date.apply(day_generate)
new_gold_data.insert(5, "tic", 'gold')
new_gold_data.insert(6, "day", gold_day)
new_gold_data.head(5)

,date,open,high,low,close,tic,day
0,2016-10-11,1259.50,1259.50,1253.45,1253.45,gold,0
1,2016-10-12,1253.45,1256.50,1253.45,1256.50,gold,1
2,2016-10-13,1256.50,1261.05,1256.50,1261.05,gold,2
3,2016-10-14,1261.05,1261.05,1251.75,1251.75,gold,3
4,2016-10-17,1251.75,1254.80,1251.75,1254.80,gold,4


In [9]:
data = pd.concat([new_bit_data, new_gold_data]).sort_values(by=['date']).reset_index(drop=True)

In [10]:
data.head()

,date,open,high,low,close,tic,day
0,2016-10-10,614.82,617.54,614.82,617.54,bitcoin,0
1,2016-10-11,1259.50,1259.50,1253.45,1253.45,gold,0
2,2016-10-11,617.54,617.54,614.77,614.77,bitcoin,1
3,2016-10-12,614.77,635.01,614.77,635.01,bitcoin,2
4,2016-10-12,1253.45,1256.50,1253.45,1256.50,gold,1


In [11]:
data.tail()

,date,open,high,low,close,tic,day
3084,2021-9-7,1821.60,1821.60,1802.15,1802.15,gold,5
3085,2021-9-8,52677.40,52677.40,46809.17,46809.17,bitcoin,2
3086,2021-9-8,1802.15,1802.15,1786.00,1786.00,gold,6
3087,2021-9-9,46809.17,46809.17,46078.38,46078.38,bitcoin,3
3088,2021-9-9,1786.00,1788.25,1786.00,1788.25,gold,0


In [12]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_vix=True,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(data)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1237, 8)
Successfully added vix


In [13]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [14]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2016-10-10,bitcoin,614.82,617.54,614.82,617.54,0.0,0.000000,620.072372,612.237628,0.000000,-66.666667,100.000000,617.540000,617.540000,13.380000
1,2016-10-11,bitcoin,617.54,617.54,614.77,614.77,1.0,-0.062147,620.072372,612.237628,0.000000,-66.666667,100.000000,616.155000,616.155000,15.360000
2,2016-10-12,bitcoin,614.77,635.01,614.77,635.01,2.0,0.548453,644.387383,600.492617,88.316146,100.000000,99.465950,622.440000,622.440000,15.910000
3,2016-10-13,bitcoin,635.01,635.96,635.01,635.96,3.0,0.847024,648.268062,603.371938,88.796573,97.825628,99.495214,625.820000,625.820000,16.690001
4,2016-10-14,bitcoin,635.96,635.96,634.02,634.02,4.0,0.914234,648.238080,606.681920,81.699760,70.604548,88.018089,627.460000,627.460000,16.120001
5,2016-10-17,bitcoin,641.42,641.42,638.97,638.97,0.0,1.402600,652.058766,611.848734,79.420756,73.083051,91.151267,631.953750,631.953750,16.209999
6,2016-10-18,bitcoin,638.97,638.97,636.29,636.29,1.0,1.273587,651.462960,613.408151,72.977957,48.436792,66.192585,632.435556,632.435556,15.280000
7,2016-10-19,bitcoin,636.29,636.29,629.25,629.25,2.0,0.800913,650.168989,614.065011,59.796152,-0.381610,21.606239,632.117000,632.117000,14.410000
8,2016-10-20,bitcoin,612.98,627.72,612.98,627.72,3.0,-0.497436,649.905369,610.406297,54.595673,-59.260044,26.865670,630.155833,630.155833,13.750000
9,2016-10-21,bitcoin,627.72,631.92,627.72,631.92,4.0,-0.311999,649.225585,611.357492,57.365101,6.088648,13.518476,630.291538,630.291538,13.340000


In [18]:
train = data_split(processed_full, '2016-10-10','2021-9-10')
trade = data_split(processed_full, '2020-10-10','2021-9-9')
print(len(train))
print(len(trade))

222
44


In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [20]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [21]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [22]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [23]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

------------------------------------
| time/                 |          |
|    fps                | 618      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.42    |
|    explained_variance | -330     |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0611   |
|    reward             | 0.0      |
|    std                | 1.01     |
|    value_loss         | 0.00466  |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 634       |
|    iterations         | 200       |
|    time_elapsed       | 1         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -1.44     |
|    explained_variance | -5.59e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 19

In [24]:
print("success")

success


In [25]:
data_risk_indicator = processed_full[(processed_full.date<'2021-9-10') & (processed_full.date>='2016-09-11')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [26]:
insample_risk_indicator.vix.describe()

count    222.000000
mean      17.086126
std        6.531776
min        9.340000
25%       12.300000
50%       14.105000
75%       21.887500
max       40.279999
Name: vix, dtype: float64

In [27]:
insample_risk_indicator.vix.quantile(0.996)

38.28216026306149

In [28]:
data.head(5)

,date,open,high,low,close,tic,day
0,2016-10-10,614.82,617.54,614.82,617.54,bitcoin,0
1,2016-10-11,1259.50,1259.50,1253.45,1253.45,gold,0
2,2016-10-11,617.54,617.54,614.77,614.77,bitcoin,1
3,2016-10-12,614.77,635.01,614.77,635.01,bitcoin,2
4,2016-10-12,1253.45,1256.50,1253.45,1256.50,gold,1


In [36]:
train

,date,tic,open,high,low,close,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2016-10-10,bitcoin,614.820000,617.540000,614.820000,617.540000,0.0,0.000000,620.072372,612.237628,0.000000,-66.666667,100.000000,617.540000,617.540000,13.380000
1,2016-10-11,bitcoin,617.540000,617.540000,614.770000,614.770000,1.0,-0.062147,620.072372,612.237628,0.000000,-66.666667,100.000000,616.155000,616.155000,15.360000
2,2016-10-12,bitcoin,614.770000,635.010000,614.770000,635.010000,2.0,0.548453,644.387383,600.492617,88.316146,100.000000,99.465950,622.440000,622.440000,15.910000
3,2016-10-13,bitcoin,635.010000,635.960000,635.010000,635.960000,3.0,0.847024,648.268062,603.371938,88.796573,97.825628,99.495214,625.820000,625.820000,16.690001
4,2016-10-14,bitcoin,635.960000,635.960000,634.020000,634.020000,4.0,0.914234,648.238080,606.681920,81.699760,70.604548,88.018089,627.460000,627.460000,16.120001
5,2016-10-17,bitcoin,641.420000,641.420000,638.970000,638.970000,0.0,1.402600,652.058766,611.848734,79.420756,73.083051,91.151267,631.953750,631.953750,16.209999
6,2016-10-18,bitcoin,638.970000,638.970000,636.290000,636.290000,1.0,1.273587,651.462960,613.408151,72.977957,48.436792,66.192585,632.435556,632.435556,15.280000
7,2016-10-19,bitcoin,636.290000,636.290000,629.250000,629.250000,2.0,0.800913,650.168989,614.065011,59.796152,-0.381610,21.606239,632.117000,632.117000,14.410000
8,2016-10-20,bitcoin,612.980000,627.720000,612.980000,627.720000,3.0,-0.497436,649.905369,610.406297,54.595673,-59.260044,26.865670,630.155833,630.155833,13.750000
9,2016-10-21,bitcoin,627.720000,631.920000,627.720000,631.920000,4.0,-0.311999,649.225585,611.357492,57.365101,6.088648,13.518476,630.291538,630.291538,13.340000


In [29]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = train, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [32]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment=e_trade_gym)
#     environment = e_trade_gym)

hit end!


In [33]:
df_account_value.shape

(222, 2)

In [34]:
pd.set_option('display.max_rows',500)
print(df_actions.actions)

0       [1]
1       [0]
2       [0]
3       [0]
4       [0]
5       [0]
6       [0]
7       [0]
8       [0]
9       [0]
10      [0]
11      [0]
12      [0]
13     [-1]
14      [1]
15      [0]
16      [0]
17      [0]
18      [0]
19      [0]
20      [0]
21      [0]
22      [0]
23      [0]
24      [0]
25      [0]
26      [0]
27      [0]
28      [0]
29      [0]
30      [0]
31      [0]
32      [0]
33      [0]
34      [0]
35      [0]
36      [0]
37      [0]
38      [0]
39      [0]
40      [0]
41      [0]
42      [0]
43      [0]
44      [0]
45      [0]
46      [0]
47      [0]
48      [0]
49      [0]
50      [0]
51      [0]
52      [0]
53      [0]
54      [0]
55      [0]
56      [0]
57      [0]
58      [0]
59      [0]
60      [0]
61      [0]
62      [0]
63      [0]
64      [0]
65      [0]
66      [0]
67      [0]
68      [0]
69      [0]
70      [0]
71      [0]
72      [0]
73      [0]
74      [0]
75      [0]
76      [0]
77      [0]
78      [0]
79      [0]
80      [0]
81      [0]
82      [0]
83  

In [35]:
df_account_value

,date,account_value
0,2016-10-10,1000.000000
1,2016-10-11,996.612460
2,2016-10-12,1016.852460
3,2016-10-13,1017.802460
4,2016-10-14,1015.862460
5,2016-10-17,1020.812460
6,2016-10-18,1018.132460
7,2016-10-19,1011.092460
8,2016-10-20,1009.562460
9,2016-10-21,1013.762460
